In [1]:
!pip install cmake 'gym[atari]' scipy

    100% |████████████████████████████████| 16.5MB 933kB/s ta 0:00:011    69% |██████████████████████▏         | 11.5MB 4.2MB/s eta 0:00:02
    100% |████████████████████████████████| 1.6MB 2.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.0MB 4.6MB/s ta 0:00:01
    100% |████████████████████████████████| 28.3MB 1.0MB/s ta 0:00:011   27% |█████████                       | 7.9MB 5.1MB/s eta 0:00:05    32% |██████████▍                     | 9.2MB 13.5MB/s eta 0:00:02    82% |██████████████████████████▍     | 23.4MB 5.4MB/s eta 0:00:01
    100% |████████████████████████████████| 2.8MB 862kB/s ta 0:00:01  2% |▉                               | 71kB 8.1MB/s eta 0:00:01
  Stored in directory: /home/manav/.cache/pip/wheels/e9/26/9b/8a1a6599a91077a938ac4348cc3d3ac84bfab0dbfddeb4c6e7
Successfully built gym
spyder 3.3.3 requires pyqt5<=5.12; python_version >= "3", which is not installed.
  Found existing installation: cloudpickle 0.8.0
    Uninstalling cloudpickle-0.8.0:
      Succ

In [3]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset()
env.render()
print("Action Space{}".format(env.action_space))
print("State Space{}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action SpaceDiscrete(6)
State SpaceDiscrete(500)


In [5]:
state=env.encode(3,1,2,0)
print("State:",state)
env.s=state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [7]:
# Without RL
env.s=328
epochs=0
penalties, reward=0,0
frames=[]
done=False
while not done:
    action=env.action_space.sample()
    state,reward,done,info=env.step(action)
    if reward==-10:
        penalties+=1
    frames.append({'frame':env.render(mode='ansi'),
                  'state':state,
                  'action':action,
                  'reward':reward})
    epochs+=1
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1952
Penalties incurred: 595


In [13]:
from IPython.display import clear_output
from time import sleep
def print_frames(frames):
    for i,frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        #print(frame['frame'].getvalue())
        print(f"Timestep: {i+1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1952
State: 0
Action: 5
Reward: 20


In [15]:
#With Q-Learning
import numpy as np
q_table=np.zeros([env.observation_space.n,env.action_space.n])

In [19]:
import random
from IPython.display import clear_output
#Hyperparameters
alpha=0.1
gamma=0.6
epsilon=0.1

#for plotting metrics
all_epochs=[]
all_penalties=[]

for i in range(1,100001):
    state=env.reset()
    epochs,penalties,reward=0,0,0
    done=False
    while not done:
        if random.uniform(0,1)<epsilon:
            action=env.action_space.sample()
        else:
            action=np.argmax(q_table[state])
        next_state, reward, done, info=env.step(action)
        
        old_value=q_table[state,action]
        next_max=np.max(q_table[next_state])
        new_value=(1-alpha)*old_value+alpha*(reward+gamma*next_max)
        q_table[state,action]=new_value
        
        if reward==-10:
            penalties+=1
        state=next_state
        epochs+=1
        
    if i%100==0:
        clear_output(wait=True)
        print(f"Episode: {i}")
print("Training finished.\n")

Episode: 100000
Training finsihed.



In [20]:
q_table[328]

array([ -2.39822645,  -2.27325184,  -2.40702552,  -2.3588204 ,
       -10.31327957, -10.06746382])

In [22]:
total_epochs, total_penalties= 0,0
episodes=100

for _ in range(episodes):
    state=env.reset()
    epochs,penalties,reward=0,0,0
    done=False
    while not done:
        action=np.argmax(q_table[state])
        state,reward,done,info=env.step(action)
        if reward==-10:
            penalties+=1
        epochs+=1
    
    total_penalties+=penalties
    total_epochs+=epochs
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.74
Average penalties per episode: 0.0
